## Создание собственного скрейпера

Библиотеки, которые нам понадобятся:

In [ ]:
# стандартные
import re
import urllib.request
from time import sleep
# нужно установить: pip install bs4
from bs4 import BeautifulSoup as bs

Посмотрим, как выглядит заглавная страница ресурса:

In [ ]:
fid = urllib.request.urlopen('https://proza.ru/')  # откроем ее с помощью библиотеки urllib

webpage = fid.read().decode('cp1251')  # как узнать, какая нам нужна кодировка: нужно открыть в браузере эту страницу, нажать f12 (для chrome) и в терминале (console) набрать document.characterSet

print(webpage)  # посмотрим, как оно выглядит

<img src="console.PNG" width=600 />

In [ ]:
soup = bs(webpage, 'html.parser')  # передадим нашу страницу в парсер bs4

In [ ]:
for link in soup.find_all('a'):
    # посмотрим, какие у нас есть ссылки
    print(link.get('href'))  

Нас интересуют только внутренние, а они, очевидно, не начинаются с https. Отсеем их:

In [ ]:
links = []
for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
    links.append('https://proza.ru' + link.get('href'))

In [ ]:
links[:10] # посмотрим, что вышло

Вроде похоже на правду. Давайте собирать уже просмотренные ссылки в множество, чтобы не попадать в них по нескольку раз:

In [ ]:
viewedlinks = set('https://proza.ru/') # главную мы типа уже посмотрели

Давайте попробуем в тестовом режиме пройтись по ссылкам до упора один раз. Давайте оформим наше извлечение страницы в какую-нибудь функцию:

In [ ]:
def looklinks(page, viewed):
    """Тестовая функция для просмотра всех ссылок на странице"""
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    soup = bs(webpage, 'html.parser') 
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    return links

In [ ]:
step = looklinks(links[0], viewedlinks)

In [ ]:
step

In [ ]:
step = looklinks(step[0], viewedlinks)
step

Если прощелкать несколько раз, рано или поздно дойдем до пустого списка: значит, ссылки на странице закончились. Давайте для проверки, что находится на страницах с авторами, прощелкаем более избирательно:

In [ ]:
# 'https://proza.ru/avtor/akochevnik'
# 'https://proza.ru/avtor/akochevnik&book=9#9'
# 'https://proza.ru/2012/11/30/1362'
# 'https://proza.ru/complain.html?text_2012/11/30/1362'
# 'https://proza.ru/about/pravila.html'

step = looklinks('https://proza.ru/2012/11/30/1362', viewedlinks)
step

Ну вроде как понятно: все должно работать. Нам теперь осталось понять, на страницах какого вида лежит хороший текст. Предположу, что на страницах с датами. Проверим, напринтив такую страницу:

In [ ]:
fid = urllib.request.urlopen('https://proza.ru/2012/11/30/1362') 
webpage = fid.read().decode('cp1251')
print(webpage)

Действительно: тут лежат наши тексты. Возможно, они еще где-нибудь лежат (poem?), это тоже можно проверить. Ну предположим, что мы все проверили и убедились, что страницы с текстами произведений - это, очевидно, адреса в формате r'\d{4}/\d\d/\d\d/.*' = '1234/56/78/123...'

Давайте попробуем допилить нашу тестовую функцию таким образом, чтобы она дополнительно проверяла, не является ли просматриваемая страница текстовой, и добавляла ее в отдельный список. 

In [ ]:
texts = []

In [ ]:
def looklinks(page, viewed, texts):
    """Тестовая функция для просмотра всех ссылок на странице и добавления текстов"""
    print(page)
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    if re.match(r'https://proza.ru/\d{4}/\d\d/\d\d/.*', page):
        texts.append(webpage)
        print(f'text for {page} added!')
    soup = bs(webpage, 'html.parser') 
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            sleep(1)  # чтоб нас случайно не заблочили как робота
            looklinks(link, viewed, texts)
    else:
        return 

Потестим нашу функцию для какого-нибудь конкретного автора:

In [ ]:
start = 'https://proza.ru/avtor/evgeny1951mai'
looklinks(start, viewedlinks, texts)


In [ ]:
texts

Видим, что все работает и что наш скрейпер, как ему и полагается, ползает со страницы на страницу и рано или поздно выкачает всю прозу, если мы не установим какой-нибудь лимит, например:

In [ ]:
def looklinks(page, viewed, texts, N=10):
    """Тестовая функция для просмотра всех ссылок на странице и добавления текстов"""
    if len(texts) > N:  # N - число текстов, которое мы хотим выкачать
        return
    print(page)
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    if re.match(r'https://proza.ru/\d{4}/\d\d/\d\d/.*', page):
        texts.append(webpage)
        print(f'text for {page} added!')
    soup = bs(webpage, 'html.parser') 
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            sleep(1)  # чтоб нас случайно не заблочили как робота
            looklinks(link, viewed, texts)
    else:
        return 

Отлично, дело за малым: вытащить собственно тексты из html-разметки :) И метаинформацию о них: заголовок, автора и дату публикации. 

Заголовок, очевидно, лежит в самом html, это:

        <title>Моё деревенское счастье. Предисловие (Александр Алексеевич Кочевник) / Проза.ру</title> 

Автор тоже есть, он зашит в заголовке: r'(.+?) \\((.+?)\\) / Проза.ру'

Ну а дату можно из ссылки страницы извлечь, мы уже выяснили, что эти ссылки нумеруются датами. Следовательно:

In [ ]:
def looklinks(page, viewed, texts, N=10):
    """Тестовая функция для просмотра всех ссылок на странице и добавления текстов"""
    print(page)
    fid = urllib.request.urlopen(page) 
    webpage = fid.read().decode('cp1251')
    viewed.add(page)
    soup = bs(webpage, 'html.parser') 
    # вот наш кусочек, который будет отвечать за добавление всего в итоговый список текстов:
    if re.match(r'https://proza.ru/\d{4}/\d\d/\d\d/.*', page):
        ta = re.search(r'(.+?) \((.+?)\) / Проза.ру', soup.title.string) # расчленяем заголовок на автора и название с помощью групп в регулярках
        if ta:
            title = ta.group(1)
            author = ta.group(2)
        else:
            # а вдруг мы где-то накосячили с регуляркой
            title = 'Unknown'
            author = 'Unknown'
        year = re.search(r'https://proza.ru/(\d{4}/\d\d/\d\d)/.*', page).group(1) # получаем год с помощью группы
        text = '\n'.join((elem.text for elem in soup.find_all("div", {"class": "text"})))  # собираем все возможные сегменты класса "текст" с помощью bs4
        texts.append({'title': title, 'author': author, 'year': year, 'text': text})
        print(f'text for {page} added!')
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            if re.search('board|login|help|topic|type|about', link): # мы, вероятно, не хотим бесконечно лазать по внутренностям не интересующих нас разделов
                continue
            if len(texts) >= N:  # N - число текстов, которое мы хотим выкачать
                return
            sleep(0.5)  # чтоб нас случайно не заблочили как робота
            looklinks(link, viewed, texts, N)
    else:
        return 

In [ ]:
viewedlinks = set('https://proza.ru/') # для чистоты эксперимента обнулим наш список просмотренного
texts = [] # и список текстов

In [ ]:
start = 'https://proza.ru/avtor/evgeny1951mai' # начнем с того же дяденьки, но укажем, что нам нужно только 10 текстов: мы потестить
looklinks(start, viewedlinks, texts, 10)

In [ ]:
texts

Ну вот и все! осталось для красоты только переписать этот код на классах и в .py....... 